# Motor Requirements

## Motor power rating

We assume here we are using this motor to power a flying drone. Using section (2.3) in source (1) introducing 
### Frames of reference
* assumption of alignment of rotor axis with e3 axis of the quadrotor
    * A: intertial frame on earth's surface
    * B: body frame of quadrotor
    * C: rotor reference frame
    * D: spinning tip plane (shaped as cone, as a consequence of rotor translation during spinning = "blade flapping"), modelled by angle β, **assumed 0 here**
* Assumption of constant propeller pitch

<div>
<img src="images/rotor_for.png" width="400"/>
</div>

### Momentum Theory (Froude)
**Assumptions:**
* Constant and equally distributed aerodynamic forces on the rotor.
* The rotor disc is infinitely thin, the air passing through it is not subject to resistance.
* Irrotational flow (no efficiency loss here).
* The flow of air outside the control volume remains undisturbed by what happens inside the CV.

These assumption are true granted $T/A$ = disc loading is large

### Momentum theory for z-axis (axial motion)

Concepts used:
* Conservation of momentum: $dm1/dt*v1 = dm2/dt*v2$)
* Mass flow rate: $dm/dt = ρ*A*V^{a}$
    * With $V_{average}$ being a vector with a e3-component only in our case
    * $A = pi*R^{2}$
* Thrust $P_{T} = T*V_{a}$ ($V_{a}$: local e3 velocity component on the disk)
* Kinetic energy: $E = (1/2)m * v²$
    * We use the derivative of the kinetic energy to time to check the kinetic power loss of the airstream inside the CV, since this will also be equal to the power used for thrust.

<div>
<img src="images/control_volume.png" width="400"/>
</div>

### End formula at hovering
At hovering, the power required for hovering is equal to: 

$P = T^{3/2} / \sqrt{2*ρ*A}$
* T: thrust required
* rho: air density
* A: propeller disc radius
* P: power in airflow

In [12]:
n_motors = 4 # (Total number of motors per drone)
m_drone = 2 # Total drone weight (kg)

In [13]:
import math
### Thrust needed
T = m_drone*9.81 / n_motors
rho = 1.2250 # kg/m³
r = 0.1 # Assume a propeller radius of about 10 cm 
A = math.pi*r**2

In [16]:
### Power calculation
P_hover = T**(3/2) / math.sqrt(2*rho*A)
print(f"Power for hovering: {P} Watt")

Power for hovering: 39.1562000844539 Watt


In [18]:
### Motor efficiency
eta = 0.8 # Typical minimum bldc motor efficiency
P_hover_r = P_hover/eta
print(f"Power given efficiency of {eta*100} %: {P_hover_r} Watt")

Power given efficiency of 80.0 %: 48.945250105567375 Watt


We take K times that number to ensure drone movability

In [21]:
K = 3
P_move_r = P_hover_r * K
print(f"End min power {P_move_r} Watt")

End min power 146.83575031670213 Watt


**Source:**
> (1) https://arxiv.org/pdf/1601.00733

## Example

### BR2826 Racerstar

* Load current: 13.8 A
* Voltage: 11.1 V
* KV: 1290 (unloaded RPM vs peak voltage) [RPM/V]

In [36]:
### INPUTS
V_MAX = 11.1
I_MAX = 13.8
KV = 1290 # in rounds per minute!
n_poles = 4

In [37]:
### Calculate the unloaded rounds per second at max load voltage
rps_mech = KV*V_MAX / 60
### The electric speed of a bldc is the mechanical speed multiplied by the number of pole pairs
rps_el = rps_mech / n_poles
### The switching cycle contains steps, so the driving frequency is the electric one * 6
rps_req = rps_el * 6
print(f"Switching frequency: {rps_req} Hz should be the minimal switching frequency of the driving signal")

Switching frequency: 357.975 Hz should be the minimal switching frequency of the driving signal


**Source:**
> https://www.racerstar.com/Racerstar-BR2826-1290KV-2-4S-Brushless-Motor-For-RC-Models-p-97.html

# Motor Modelling

## Electrical
<div>
<img src="images/bldc_windings.png" width="600"/>
</div>

* Ls: self-inductance of the phase winding
* M: mutual inductance
* Vu, Vv, Vw: phase voltage referred to virtual ground
* ea, eb, ec: back-emf as a consequence of resistance against magnetic field change due to movement
* ia, ib, ic: phase currents, assume ia+ib+ic = 0
<div>
<img src="images/emf_equations.png" width="600"/>
</div>

### Trapezoidal EMF
The EMF is assumed ot be trapezoidal due to bldc construction. \
$e_{a} = \omega_{m} * \lambda_{m} * f_{as}(\theta_{r})$ \
$e_{b} = \omega_{m} * \lambda_{m} * f_{bs}(\theta_{r})$ \
$e_{c} = \omega_{m} * \lambda_{m} * f_{cs}(\theta_{r})$ 

* $\omega_{m}$: angular rotor speed in radians per second
* $\lambda_{m}$: flux linkage
* $\theta_{r}$: mechanical rotor position in radians
* functions $f_{is}$ are normalized trapezoidal functions (max(|f| = 1)

### Torque
Equalling the electromagnetic power and the resulting electromagnetic torque gives us: \
=> $P_{e}=e_{a}*i_{a}+e_{b}*i_{b}+e_{c}*i_{c}$ \
=> $P_{m} = T_{e} * \omega_{m}$ \
<=> $T_{e} = [e_{a}*i_{a}+e_{b}*i_{b}+e_{c}*i_{c}] / \omega_{m}$ 


The moment of inertia as well as the torques resulting from them can be divided into 
* load inertia $J_{L}, T_{L}$
* machine inertia $J_{M}, T_{M}$.

### Newton
Given inertia J, friction coefficient B, the mechanical system can be described by ($\alpha = d\omega/dt$):

$J*\alpha_{m} + B*\omega_{m} = (T_{e} - T_{L})$

### Electrical vs Mechanical speed
The electrical speed is equal to the mechanical speed times the amount of pole pairs (poles / 2), since every electrical period needs to be repeated p/2 number of times for 1 single mechanical revolution (2*$\pi$)

$\frac{d\theta_{r}}{dt} = \frac{P}{2}*\omega_{m}$

## 5 Final equations

### currents
$\frac{di_{a}}{dt} = - [\frac{R_{s}}{L-M}] * i_{a} - [(\frac{\lambda_m}{J}) * f_{as}(\theta_{r})] * \omega_{m} + [\frac{1}{L-M}]*V_{as} - [\frac{1}{L-M}]*e_{a}$ \
$\frac{di_{b}}{dt} = - [\frac{R_{s}}{L-M}] * i_{b} - [(\frac{\lambda_m}{J}) * f_{bs}(\theta_{r})] * \omega_{m} + [\frac{1}{L-M}]*V_{bs} - [\frac{1}{L-M}]*e_{b}$ \
$\frac{di_{c}}{dt} = - [\frac{R_{s}}{L-M}] * i_{c} - [(\frac{\lambda_m}{J}) * f_{cs}(\theta_{r})] * \omega_{m} + [\frac{1}{L-M}]*V_{cs} - [\frac{1}{L-M}]*e_{c}$

### Mechanical speed

$\frac{d\omega_{m}}{dt} = [(\frac{\lambda_m}{J}) * f_{as}(\theta_{r})] * i_{a} + [(\frac{\lambda_m}{J}) * f_{bs}(\theta_{r})] * i_{b}  + [(\frac{\lambda_m}{J}) * f_{cs}(\theta_{r})] * i_{c} - [\frac{B}{J}] * \omega_{m}$

### Electrical angle
$\frac{d\theta_{r}}{dt} = \frac{P}{2}*\omega_{m}$

**Sources:**
> https://core.ac.uk/download/pdf/53188902.pdf